#Import

In [ ]:
import tensorflow as tf
import numpy as np
import PIL
import matplotlib.pyplot as plt
import pathlib
import os
from pathlib import Path
from PIL import Image
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50

#Mount Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#Import Kaggle

In [ ]:
from google.colab import files
files.upload()

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d agungmrf/indonesian-sign-language-bisindo

Saving kaggle.json to kaggle.json
rm: cannot remove '/root/.kaggle': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/agungmrf/indonesian-sign-language-bisindo
License(s): unknown
100% 1.36G/1.36G [00:55<00:00, 30.9MB/s]
100% 1.36G/1.36G [00:55<00:00, 26.4MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('indonesian-sign-language-bisindo.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

#Directory

In [ ]:
img_width = 224
img_height = 224

In [ ]:
data_train_path = 'bisindo/images/train'
data_val_path = 'bisindo/images/val'

In [ ]:
data_train = tf.keras.utils.image_dataset_from_directory(
    data_train_path,
    shuffle=True,
    image_size=(img_width, img_height),
    batch_size=32,
    validation_split=None
)

data_val = tf.keras.utils.image_dataset_from_directory(
    data_val_path,
    shuffle=False,
    image_size=(img_width, img_height),
    batch_size=32,
    validation_split=None
)

#Image Data Generator (ga kepake)

In [ ]:
# train_datagen = ImageDataGenerator(
#     rescale=1./255.,
#     rotation_range=0.40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     fill_mode='nearest',
#     horizontal_flip=True,
# )

# val_datagen = ImageDataGenerator(
#     rescale=1./255.

# )

In [ ]:
# train_generator = train_datagen.flow_from_directory(
#     train_images_path,
#     target_size = (224,224),
#     class_mode='categorical',
#     batch_size=126
# )

# val_generator = val_datagen.flow_from_directory(
#     val_images_path,
#     target_size = (224,224),
#     class_mode='categorical',
#     batch_size=126
# )

#Model

In [ ]:
# Define image dimensions
img_width = 224
img_height = 224

# Define paths to training and validation data
data_train_path = 'bisindo/images/train'
data_val_path = 'bisindo/images/val'

# Load training and validation data using tf.keras.utils.image_dataset_from_directory
data_train = tf.keras.utils.image_dataset_from_directory(
    data_train_path,
    shuffle=True,
    image_size=(img_width, img_height),
    batch_size=32,
    validation_split=None  # Set to None since you are not using validation split
)

data_val = tf.keras.utils.image_dataset_from_directory(
    data_val_path,
    shuffle=False,
    image_size=(img_width, img_height),
    batch_size=32,
    validation_split=None  # Set to None since you are not using validation split
)

# Define number of classes and build ResNet50 model
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T','U', 'V', 'W', 'X', 'Y', 'Z']
num_classes = len(classes)

def build_resnet50(num_classes, input_shape=(224, 224, 3)):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the model
model = build_resnet50(num_classes)

# Train the model
model.fit(
    data_train,
    epochs=10,
    validation_data=data_val
)

Found 9169 files belonging to 26 classes.
Found 2301 files belonging to 26 classes.
94765736/94765736 [==============================] - 3s 0us/step


Epoch 1/10
287/287 [==============================] - 145s 360ms/step - loss: 0.6286 - accuracy: 0.8284 - val_loss: 1.4180 - val_accuracy: 0.7227
Epoch 2/10
287/287 [==============================] - 99s 342ms/step - loss: 0.1577 - accuracy: 0.9508 - val_loss: 0.4403 - val_accuracy: 0.8657
Epoch 3/10
287/287 [==============================] - 100s 346ms/step - loss: 0.0980 - accuracy: 0.9673 - val_loss: 0.2612 - val_accuracy: 0.9231
Epoch 4/10
287/287 [==============================] - 101s 350ms/step - loss: 0.0721 - accuracy: 0.9778 - val_loss: 2.0875 - val_accuracy: 0.6767
Epoch 5/10
287/287 [==============================] - 101s 350ms/step - loss: 0.0571 - accuracy: 0.9819 - val_loss: 0.2531 - val_accuracy: 0.9322
Epoch 6/10
287/287 [==============================] - 101s 348ms/step - loss: 0.0772 - accuracy: 0.9769 - val_loss: 0.8916 - val_accuracy: 0.7579
Epoch 7/10
287/287 [==============================] - 101s 351ms/step - loss: 0.0429 - accuracy: 0.9867 - val_loss: 0.2729 - 

In [ ]:
#Accuracy plot
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

#Loss plot
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
saved_model_path = "./bisindo_model.h5"

# YOUR CODE HERE
import time
model.save(saved_model_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 116.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 112.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 24.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
  

In [ ]:
!tensorflowjs_converter --input_format keras bisindo_model.h5 model

2024-06-01 11:20:55.201588: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 11:20:56.270045: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!zip -r /content/model.zip /content/model

  adding: content/model/ (stored 0%)
  adding: content/model/group1-shard3of25.bin (deflated 7%)
  adding: content/model/group1-shard8of25.bin (deflated 7%)
  adding: content/model/group1-shard1of25.bin (deflated 7%)
  adding: content/model/group1-shard16of25.bin (deflated 7%)
  adding: content/model/group1-shard10of25.bin (deflated 7%)
  adding: content/model/group1-shard2of25.bin (deflated 7%)
  adding: content/model/group1-shard20of25.bin (deflated 7%)
  adding: content/model/group1-shard18of25.bin (deflated 7%)
  adding: content/model/group1-shard17of25.bin (deflated 7%)
  adding: content/model/group1-shard15of25.bin (deflated 7%)
  adding: content/model/group1-shard23of25.bin (deflated 7%)
  adding: content/model/group1-shard24of25.bin (deflated 7%)
  adding: content/model/group1-shard4of25.bin (deflated 7%)
  adding: content/model/group1-shard7of25.bin (deflated 7%)
  adding: content/model/group1-shard25of25.bin (deflated 7%)
  adding: content/model/group1-shard9of25.bin (deflate